## Isolation Forest
iForest用于异常检测
论文地址：https://cs.nju.edu.cn/zhouzh/zhouzh.files/publication/icdm08b.pdf

基于两个假设：
1. 异常数据是少量的；
2. 异常数据属性分布和正常数据属性分布不同
根据上面两个假设，可以得到一个推论：通过隔离空间，可以较快的将异常数据隔离出来。

相比于传统的基于距离/密度的基于模型方法，iForest有以下几个优点：
1. iTrees的isolation特性能够建立partial models(下面会说明)，并且对子样本数据有更加充分的利用，这都是现有方法无法做到的；
2. iForest不需要进行距离/密度计算，而这两个往往意味着很多的计算量；
3. iForest有线性复杂度和比较低的内存需求，而现有的方法中最好的也才近似到线性复杂并且有很大的内存依赖；
4. iForest有更好的扩展性，包括大数据两和大量无关的属性。



isolation就是将一个instance和其他的instances分开，迭代分割直到所有实例被独立。

一个随机划分对异常数据产生更短的分割路径，因为：
1. 越少的异常数据会产生越少的划分（在树结构中有更少的路径）；
2. 拥有特别的属性值更可能被更早的划分开


随机选择一个属性，并在属性的最大最小值中间随机选择一个值作为分割。因为是随机的选择（属性和分割点），所以可以进行多次，然后计算平均值，得到一个收敛结果。

#### 定义

__Isolation Tree：__
假设T是一个isolation tree的节点，那么T要么是一个叶子节点（外部节点，没有子节点），要么是一个内部节点。这个内部节点有两个子女节点($T_l,T_r$)，并且有一个test，这个test由p和q组成，q为一个属性，而p为这个属性的分割点。q<p将数据分割成$T_l$和$T_r$。

所得到的数是完全二叉树，即树节点要么没有子节点，要么就有两个子节点。在完全分割的情况下，有n个叶子节点（n个数据），则有n-1个父节点，则总共有2n-1个节点，所以内存是线性随n增长。一般停止分割的原因：
1. 树到达高度限制；
2. |X| = 1，已经全部被隔离；
3. 剩下的数据都有相同的值

树建好后，异常检测的任务就是根据路径长度（异常分数anomaly score）来对节点排序。从root节点开始计算。

__Path Length:__
从root到某个外部节点所需要的遍历的节点数，记作h(x)，x为某个数据表示把这个数据分到某个叶子节点需要的步骤。
任何异常检测都需要一个anomaly score，但是这里的的anomaly score很明显依赖数据量n（因为n的大小决定这个分数的上限，而且均值为$\log n$，也取决于n)。因为h(x)不受限制且不容易比较，所以采用以下方法计算一个score。

__Anomaly Score:__
Anomaly Score记作s(x, n), 通过对iTree和BST的比对，

|iTree|BST|
|-----|---|
|Proper binary trees|Proper binary trees|
|External node termination|Unsuccessful search|
|Not applicable|Successful search|

使用：
$$ c(n) = 2H(n-1) -(2(n-1)/n)\\ H(i) == ln(i) + 0.5772156649(Euler's constant)\\
s(x, n) = 2^{-(E(h(x))/c(n)}
$$
$E(h(x))$是$h(x)$iTree的集合（因为随机查找，所以可以尝试多次，求平均值）的平均数。对与上式：
1. $E(h(x))\to c(n), s\to0.5$
2. $E(h(x))\to 0, s\to1$
3. $E(h(x))\to n - 1, s\to0$

通过异常分数可以得到以下判断：
1. 如果s接近1，则这个数据基本上就是异常数据；
2. 如果s值小于0.5，则更倾向是正常值；
3. 如果所有数据的s约等于0.5，那么全部样例应该就没有异常数据

#### iForest
iForest作为一个提升方法，有：
1. 通过更短的path length来标识异常点；
2. 多个树来标识不同的异常点。

iForest可以不需要将大多数normal point隔离开（partial model），使用一个小的样本集合来创建模型。

因为Swamping和Masking的原因，数据太多就会导致误分类增大或者漏掉的增多，所以使用抽样得到子样本集来运行。一般使用无放回抽样获得子集

### 算法
包括训练和评估两个阶段，也是无监督算法。

#### train
分作两个方面介绍，iForest和iTree。
##### iForest
Algorithm 1 : iF orest(X, t, ψ)
```python
Inputs: X - input data, t - number of trees, ψ - sub-
sampling size
Output: a set of t iTrees
1: Initialize F orest
2: set height limit l = ceiling(log 2 ψ)
3: for i = 1 to t do
4:    X 0 ← sample(X, ψ)
5:    Forest ← F orest ∪ iT ree(X 0 , 0, l)
6: end for
7: return F orest
```
##### iTree
Algorithm 2 : iT ree(X, e, l)
```python
Inputs: X - input data, e - current tree height, l - height
limit
Output: an iTree
1: if e ≥ l or |X| ≤ 1 then
2:    return exNode{Size ← |X|}
3: else
4:    let Q be a list of attributes in X
5:    randomly select an attribute q ∈ Q
6:    randomly select a split point p from max and min
         values of attribute q in X
7:    X_l ← filter(X, q < p)
8:    X_r ← filter(X, q ≥ p)
9:    return inNode{Left ← iT ree(X_l , e + 1, l),
10:                 Right ← iT ree(X_r , e + 1, l),
11:                 SplitAtt ← q,
12:                 SplitValue ← p}
13: end if
```
#### 评估
Algorithm 3 : P athLength(x, T, e)
```python
Inputs : x - an instance, T - an iTree, e - current path length;
to be initialized to zero when first called
Output: path length of x
1: if T is an external node then
2:    return e + c(T.size) {c(.) is defined in Equation 1}
3: end if
4: a ← T.splitAtt
5: if x a < T.splitV alue then
6:    return P athLength(x, T.lef t, e + 1)
7: else {x a ≥ T.splitV alue}
8:    return P athLength(x, T.right, e + 1)
9: end if
```

__名词：__
1. Swamping：被误分类到异常点的正常数据，正常数据和异常数据比较靠近，所以path length增加；
2. Masking:异常数据比较多，而且比较密度较大时，需要隔离开的path length往往也比较大了。